This code is part of a preprocessing step for static layers used in firenet. It involves loading various landfire layers, reprojecting them to a common coordinate reference system (CRS), and then stacking them together into a single multi-layered dataset. Landfire data gives spatial information such as elevation, vegetation height etc. that are useful to feed into firenet when it makes inference via interpreting spectral satellite data.

The reprojected layers are matched to the CRS of a median image to ensure spatial alignment across all layers. This combined dataset is then saved as a NetCDF file for further use. 

Memory management is also considered by manually invoking garbage collection after processing each layer to ensure the environment won't crash.


In [1]:
import gc
import xarray as xr
import rioxarray

# Load the landfire layers
landfire_tif_paths = ['/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVC_220_CONUS/Tif/LC22_EVC_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_EVH_220_CONUS/Tif/LC22_EVH_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FBFM40_220_CONUS/Tif/LC22_F40_220.tif',
                        '/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_FVH_220_CONUS/Tif/LC22_FVH_220.tif']

# Initialize an empty dictionary to store the datasets
datasets = {}

In [2]:
import rioxarray
from rasterio.enums import Resampling
import os

# Define the target resolution
# Create an empty dictionary to hold the reprojected/resampled datasets
landfire_datasets = {}

median_ds = rioxarray.open_rasterio('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/reprojected_median_ds.nc')

In [3]:
median_ds.rio.crs

CRS.from_epsg(5070)

In [4]:
# Loop through each Landfire TIFF file path
for tif_path in landfire_tif_paths[0:]:
    print(tif_path)
    # Open the TIFF file as an xarray dataset
    landfire_ds = rioxarray.open_rasterio(tif_path, chunks={'x': 1024, 'y': 1024})
    
    # Define a bounding box in the western US
    # bounding box coordinates in epsg: [minx, miny, maxx, maxy]
    bbox = [-2349705.0, 1844810.0, -1500000.0, 3159255.0]


    # Print the original CRS of the dataset
    print(f"Original CRS: {landfire_ds.rio.crs}")

    # Print the bounds of the original dataset
    print(f"Original bounds: {landfire_ds.rio.bounds()}")

    # Clip the dataset to the bounding box before reprojecting
    landfire_clipped = landfire_ds.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])

    # Print the bounds of the clipped dataset
    print(f"Clipped bounds: {landfire_clipped.rio.bounds()}")    # Clip the dataset to the bounding box before reprojecting
    landfire_clipped = landfire_ds.rio.clip_box(minx=bbox[0], miny=bbox[1], maxx=bbox[2], maxy=bbox[3])

    # Reproject the clipped dataset to the target resolution
    landfire_reprojected = landfire_clipped.rio.reproject(
        "EPSG:5070",
        resolution=375,
        resampling=Resampling.bilinear  # Choose an appropriate resampling method
    )

    # Print the CRS after reprojection
    print(f"Reprojected CRS: {landfire_reprojected.rio.crs}")

    # Print the bounds of the reprojected dataset
    print(f"Reprojected bounds: {landfire_reprojected.rio.bounds()}")

    # Print the resolution of the reprojected dataset
    print(f"Reprojected resolution: {landfire_reprojected.rio.resolution()}")

    # Print the shape of the reprojected dataset
    print(f"Reprojected shape: {landfire_reprojected.rio.shape}")

    del landfire_ds
    gc.collect()  # Invoke garbage collection

    # Get the variable name from the TIFF file name
    var_name = os.path.splitext(os.path.basename(tif_path))[0]
    
    # Define the new file name
    new_file_name = f"/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/{var_name}_reprojected.tif"
    # Save the reprojected/resampled dataset as a new TIFF file
    landfire_reprojected.rio.to_raster(new_file_name)
    
    # Close the dataset
    landfire_reprojected.close()
    del landfire_reprojected
    gc.collect()  # Invoke garbage collection

/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2020_Elev_220_CONUS/Tif/LC20_Elev_220.tif
Original CRS: EPSG:5070
Original bounds: (-2362395.0, 221265.0, 2327655.0, 3267405.0)
Clipped bounds: (-2349705.0, 1844805.0, -1499985.0, 3159255.0)
Reprojected CRS: EPSG:5070
Reprojected bounds: (-2349705.0, 1844505.0, -1499955.0, 3159255.0)
Reprojected resolution: (375.0, -375.0)
Reprojected shape: (3506, 2266)
/Users/adamhunter/Documents/school projs/firenet/data/static_layers/LF2022_CBD_220_CONUS/Tif/LC22_CBD_220.tif
Original CRS: EPSG:5070
Original bounds: (-2362395.000000001, 221265.00000000373, 2327654.999999999, 3267405.0000000037)
Clipped bounds: (-2349705.000000001, 1844805.0000000037, -1499985.000000001, 3159255.0000000037)
Reprojected CRS: EPSG:5070
Reprojected bounds: (-2349705.000000001, 1844505.0000000037, -1499955.000000001, 3159255.0000000037)
Reprojected resolution: (375.0, -375.0)
Reprojected shape: (3506, 2266)
/Users/adamhunter/Documents/school projs/fire

In [11]:
import glob
import rioxarray
import os
import xarray as xr

# Get a list of all reprojected TIFF files
reprojected_tif_paths = glob.glob('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/*_reprojected.tif')

# Initialize an empty dictionary to store the datasets
datasets = {}

# Load the first dataset as the reference
reference_ds = rioxarray.open_rasterio(reprojected_tif_paths[0])

# Loop through each reprojected TIFF file path
for tif_path in reprojected_tif_paths:
    # Open the TIFF file as an xarray dataset
    ds = rioxarray.open_rasterio(tif_path)
    
    # Reproject the dataset to match the reference dataset
    ds_matched = ds.rio.reproject_match(reference_ds)
    
    # Get the variable name from the TIFF file name
    var_name = os.path.splitext(os.path.basename(tif_path))[0].replace('_reprojected', '')
    
    # Add the matched dataset to the dictionary, using the variable name as the key
    datasets[var_name] = ds_matched

# Combine all matched datasets into a single xarray dataset
combined_ds = xr.Dataset(datasets)

# Save the combined dataset as a NetCDF file
combined_ds.to_netcdf('/Users/adamhunter/Documents/school projs/firenet/data/static_layers/landfire reprojected/combined_landfire.nc')